In [1]:
#imports
from selenium import webdriver 

In [2]:
#Driver Path
DRIVER_PATH = '/Volumes/Data/Softwares/ChromeDriver/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
'''
open the website
=================
Step 1: Confirm you are human and click on continue
Step 2: Click on the dataset
Step 3: Go to active window
Step 4: Get all the links for titles
'''
driver.get('https://trials.proquest.com/access?token=wyBHSUeurwOlxBgwkzELezMTq')

In [4]:
'''
for handle in driver.window_handles:
    driver.switch_to.window()
    print(driver.current_url)
'''
driver.switch_to.window(driver.window_handles[0])

In [ ]:
'''
Get results links from each page
'''
def getTitleLinksPerPage(linksArray):
    print('Getting Title Links..')
    searchResults = driver.find_element_by_id('search-results')
    linkTags =  searchResults.find_elements_by_class_name('search-result-title-link')
    for linkTag in linkTags:
        linksArray.append(linkTag.get_attribute('href'))
    return linksArray

def gotoNextPage():
    if(len(driver.find_elements_by_class_name('pager-next'))):
        print('Move To next page..')
        nextPager = driver.find_element_by_class_name('pager-next')
        nextPager.find_element_by_partial_link_text("next").click()
        return True
    else:
        return False
    
def getAllTitleLinks(linksArr):
    linksArr = getTitleLinksPerPage(linksArr)
    if(gotoNextPage()):
        getAllTitleLinks(linksArr)
    return linksArr

In [ ]:
linksArr = []
getAllTitleLinks(linksArr)
print(len(linksArr))

In [ ]:
import json 
tempData = { 'links': linksArr }
with open('../linksData.json', 'w') as outfile:
    json.dump(tempData, outfile)

In [5]:
import json
with open('../linksData.json') as f:
    linksArr = json.load(f)['links']

In [17]:
import json
from time import sleep
class Transcript:
    def __init__(self):
        self.viewer = None
        self.viewerToolbar = None
        self.viewerMainLayout = None
        self.viewerSideBarContent = None
        self.viewerMainContent = None


    def getSideBarData(self):
        data = {}
        data['Title'] = self.viewerSideBarContent.find_element_by_tag_name('a').get_attribute('innerHTML')
        dlTag = self.viewerSideBarContent.find_element_by_tag_name('dl')
        dtTags = dlTag.find_elements_by_tag_name('dt')
        ddTags = dlTag.find_elements_by_tag_name('dd')
        for i in range(len(dtTags)):
            data[dtTags[i].get_attribute('innerHTML')] = ddTags[i].find_element_by_tag_name('span').get_attribute('innerHTML')
        return data

    def getTranscript(self):
        data = {}
        pages = self.viewerMainContent.find_elements_by_class_name('ucv-text-page')
        for page in pages:
            data['Title'] = page.find_element_by_class_name('ucv-section-title').get_attribute('innerHTML')
            data['conversation'] = []
            transcript = page.find_element_by_class_name('text-page')
            contentArr = transcript.find_elements_by_tag_name('p')
            keyWordSet = set(['TRANSCRIPT', 'TRANSCRIPT:', 'CLIENT', 'CLIENT:', 'THERAPIST', 'THERAPIST:', 'BEGIN', 'END'])
            for itm in contentArr:
                if(itm.find_elements_by_tag_name('span')):
                    text = itm.find_element_by_tag_name('span').get_attribute('innerHTML')
                    if(len(set(text.split()).intersection(keyWordSet))):
                        data['conversation'].append(text)
        return data
    
    def writeJsonToFile(self, name, data):
        with open(name+'.json', 'w') as outfile:
            json.dump(data, outfile)

    def getAllTranscript(self, arrayLinks):
        for index, link in enumerate(arrayLinks):
            i = index + 1634
            driver.get(link)
            sleep(10)
            self.viewer = driver.find_element_by_id('ucv-viewer')
            self.viewerToolbar = self.viewer.find_element_by_class_name('rpv-default-layout-toolbar')
            self.viewerMainLayout = self.viewer.find_element_by_class_name('rpv-default-layout-main')
            self.viewerSideBarContent = self.viewerMainLayout.find_element_by_class_name('rpv-default-layout-sidebar-content')
            self.viewerMainContent = self.viewerMainLayout.find_element_by_id('ucv-pages-viewer')
            
            sideData = self.getSideBarData()
            mainData = self.getTranscript()
            
            data = { 'Info': sideData, 'data': mainData }
            self.writeJsonToFile(str(i),data)  

In [18]:
transcript = Transcript()
transcript.getAllTranscript(linksArr[1634:])